<a href="https://colab.research.google.com/github/Tahimi/TriangulationBestPathModel/blob/main/triangulacao1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from osgeo import gdal
import numpy as np

In [ ]:
### Define the Elevation Field ###

# read matrix from tif file
ds = gdal.Open(r'../ROI/ROI_topodata.tif')
elevation = np.matrix(ds.GetRasterBand(1).ReadAsArray())
elevation = elevation.T # [columns, lines]
ds = None

# save the elevation matrix into a text file
#np.savetxt(r'elevation.csv', elevation, delimiter=',', fmt='%6d')
#print(elevation.shape)

# ignoring the tif file, and using a constante elevation field for testing
#'A' cell selected,  [ 97 248]
#'B' cell selected,  [229  92]
#elevation = np.ones(shape=((229-97)//8, (248-92)//8), dtype=float)  # shape=(columns, lines)

# print shape and min and max values of elevation matrix
print(elevation.shape)
print('elevation min and max values = ', np.min(elevation), ', ', np.max(elevation))

In [ ]:
### Define The Grid Parameters ###

#!pip install screeninfo
from screeninfo import get_monitors
screen = get_monitors()[0]
print('screen.width = ', screen.width, ', screen.height = ', screen.height)

# define grid parameters
gridParameters = {'Pixel':{'sizeInMeters':np.array([30.,30.])}}

def updateGridParameters(originalGridSize, coarseningFactor, screenOccupationCoeff):
    gridParameters.update( {'Cell':{'sizeInPixels':coarseningFactor}} )
    gridParameters['Cell'].update( {'sizeInMeters':gridParameters['Cell']['sizeInPixels'] * gridParameters['Pixel']['sizeInMeters']} )
    
    gridParameters.update( {'Grid':{'sizeInPixels':originalGridSize}} )
    gridParameters['Grid'].update( {'sizeInIdx':(gridParameters['Grid']['sizeInPixels'] / gridParameters['Cell']['sizeInPixels']).astype(int)} )
    gridParameters['Grid'].update( {'sizeInMeters':gridParameters['Grid']['sizeInIdx'] * gridParameters['Cell']['sizeInMeters']} )
    
    windowMaxOcupationInScreenPixels = np.array([screen.width, screen.height]) * screenOccupationCoeff
    screenPixelsPerMeter = min(windowMaxOcupationInScreenPixels / gridParameters['Grid']['sizeInMeters'])
    cellSizeInScreenPixels = (screenPixelsPerMeter * gridParameters['Cell']['sizeInMeters']).astype(int)

    windowSizeInScreenPixels = cellSizeInScreenPixels * gridParameters['Grid']['sizeInIdx']

    gridParameters['Grid'].update({'sizeInScreenPixels':windowSizeInScreenPixels})
    gridParameters['Cell'].update({'sizeInScreenPixels':cellSizeInScreenPixels})

# update grid parameters
updateGridParameters(elevation.shape, np.array([1, 1]), 0.9)

# print grid parameters
import pprint
pprint.pprint(gridParameters)

In [ ]:
### Define the Grid ###

class Cell:
    def __init__(self, Idx, grid):
        self.Idx = np.array(Idx) # [columnIdx, lineIdx]
        self.grid = grid
        self.data = {'elevation': 0.}
        self.centerIdx = self.Idx + np.array([1/2.,1/2.])

    def __eq__(self, other):
        return np.array_equal(self.Idx, other.Idx)

    def elevation(self):
        return self.data['elevation']

    def columnIdx(self):
        return self.Idx[0]

    def lineIdx(self):
        return self.Idx[1]

    def positionInScreenPixels(self):
        return self.grid.positionInScreenPixelsFromIdx(self.Idx)
    
    def positionInMeters(self):
        return self.grid.positionInMetersFromIdx(self.Idx)

    def centerPositionInScreenPixels(self):
        return self.grid.positionInScreenPixelsFromIdx(self.centerIdx)
    
    def centerPositionInMeters(self):
        return self.grid.positionInMetersFromIdx(self.centerIdx)

class Grid:
    def __init__(self, gridParameters):
        self.gridParameters = gridParameters

        self.cellSizeInMeters = self.gridParameters['Cell']['sizeInMeters']
        self.cellSizeInScreenPixels = self.gridParameters['Cell']['sizeInScreenPixels']

        self.sizeInMeters = self.gridParameters['Grid']['sizeInMeters']
        self.sizeInScreenPixels = self.gridParameters['Grid']['sizeInScreenPixels']
        self.dim = self.gridParameters['Grid']['sizeInIdx'] # [columnsNbr, linesNbr]

        # instanciating the grid cells
        self.makeGridCells_()
        
        # seting the elevation value for each grid cell
        self.elevationIsSet = False
        self.setElevation()

    def infos(self):
        print('Grid dimension, [columnsNbr, linesNbr] = ', self.dim)

    def makeGridCells_(self):
        self.cells = np.ndarray(shape = self.dim, dtype=Cell)
        for j in range(self.columnsNbr()):
            for i in range(self.linesNbr()):
                self.cells[j,i] = Cell([j,i], self)

    def columnsNbr(self):
        return self.dim[0]

    def linesNbr(self):
        return self.dim[1]

    def cellsNbr(self):
        return self.columnsNbr() * self.linesNbr()

    def columnFromCell(self, cell):
        columnIdx = cell.columnIdx()
        return self.cells[columnIdx,:]

    def lineFromCell(self, cell):
        lineIdx = cell.lineIdx()
        return self.cells[:,lineIdx]

    def cellFromIdx(self, idx):
        columnIdx = int(idx[0]); lineIdx = int(idx[1])
        return self.cells[columnIdx, lineIdx]

    def positionInScreenPixelsFromIdx(self, idx):
        return (idx * self.cellSizeInScreenPixels).astype(int)
    
    def positionInMetersFromIdx(self, idx):
        positionIn2D = idx * self.cellSizeInMeters
        #return np.append(positionIn2D, 0.) # adim horizontal projection
        return np.append(positionIn2D, self.cellFromIdx(idx).elevation())

    def idxFromPositionInMeters(self, positionInMeters):
        positionIn2D = positionInMeters[:2]
        return positionIn2D / self.cellSizeInMeters

    def idxFromPositionInScreenPixels(self, positionInScreenPixels):
        return positionInScreenPixels / self.cellSizeInScreenPixels

    def cellIdxFromIdx(self, idx):
        cell = self.cellFromIdx(idx)
        return cell.Idx

    def positionInScreenPixelsFromCellIdx(self, idx):
        idx = self.cellIdxFromIdx(idx)
        return self.positionInScreenPixelsFromIdx(idx)
    
    def cellFromPositionInMeters(self, positionInMeters):
        idx = self.idxFromPositionInMeters(positionInMeters)
        return self.cellFromIdx(idx)
    
    def cellFromPositionInScreenPixels(self, positionInScreenPixels):
        idx = self.idxFromPositionInScreenPixels(positionInScreenPixels)
        return self.cellFromIdx(idx)
    
    def cellsIdx(self, cells):
        listOfIdxs = []
        for cell in cells:
            listOfIdxs.append(cell.Idx)
        return listOfIdxs

    def setElevation(self):
        for j in range(self.columnsNbr()):
            for i in range(self.linesNbr()):
                self.cells[j,i].data['elevation'] = elevation[j,i]
        self.elevationIsSet = True

# instanciate the Grid
grid = Grid(gridParameters)
grid.infos()

In [ ]:
### Define the Displayer, (using pygame package) ###

#!pip install pygame
import pygame as pg
from pygame.locals import *

# displaying this image in the background helps selecting the A and B points
backgroundImage = r'../ROI/ROI_topodata_vf.png'

class DisplayManager:
    def __init__(self, grid, windowCaptionParte1 = f'Triangulacao'):
        self.grid = grid
        self.windowCaptionParte1 = windowCaptionParte1

        self.windowDim = self.grid.gridParameters['Grid']['sizeInScreenPixels']
        self.windowWidth = self.windowDim[0]
        self.windowHeight = self.windowDim[1]

        self.A = None; self.AIsSet = False
        self.B = None; self.BIsSet = False
        self.solver = None; self.solverIsSet = False
        self.updateDataSpan_ = None
        
        self.backgroundImage = pg.image.load(backgroundImage)
        self.backgroundImage = pg.transform.scale(self.backgroundImage, self.windowDim)
        self.initiateDisplay()
        
    def initiateDisplay(self):
        pg.init()
        pg.font.init()
        
        self.window = pg.display.set_mode(self.windowDim)
        pg.display.set_caption(self.windowCaptionParte1)
        self.window.blit(self.backgroundImage, [0,0])

        self.clock = pg.time.Clock()
        self.clock.tick()
        self.display = True
        self.updating = True
        self.iter = -1

    def problemIsSet(self):
        return self.grid.elevationIsSet and self.AIsSet and self.BIsSet

    def setAFromIdx(self, idx):
        self.A = grid.cellFromIdx(idx)
        self.grid.A = self.A
        self.AIsSet = True
        print("'A' cell selected, ", self.A.Idx)

    def setBFromIdx(self, idx):
        self.B = grid.cellFromIdx(idx)
        self.grid.B = self.B
        self.BIsSet = True
        print("'B' cell selected, ", self.B.Idx)

    def setSolver(self, solver):
        self.solver = solver
        self.solverIsSet = True

    def update(self):
        self.draw()
        #self.printData()
        pg.display.set_caption(self.windowCaption())
        pg.display.flip()
        self.processEvent()
        
    def draw(self):
        if self.problemIsSet():
            #self.drawElevationField()
            self.drawCell(self.A, pg.Color('green'), True)
            self.drawCell(self.B, pg.Color('red'), True)
            self.drawLine(pg.Color('blue'), self.A, self.B)
            #self.drawGrid()

        if self.solverIsSet:
            self.drawLines(pg.Color('darkmagenta'), self.solver.bestPathCells)

    def printData(self):
        if not self.grid.elevationIsSet: return
        self.printElevationField()
        
    def drawGrid(self):
        for j in range(self.grid.columnsNbr()):
            for i in range(self.grid.linesNbr()):
                cell = self.grid.cellFromIdx([j,i])

                self.drawCell(cell)

    def drawElevationField(self, minColor = pg.Color('black'), maxColor = pg.Color('white')):
        if not self.grid.elevationIsSet: return
        self.updateDataSpan()

        for j in range(self.grid.columnsNbr()):
            for i in range(self.grid.linesNbr()):
                cell = self.grid.cellFromIdx([j,i])

                dataValue = cell.elevation()
                if self.spanOfData != 0:
                    coeff = (dataValue - self.minExposedDataValue) / self.spanOfData
                    color = minColor.lerp(maxColor, coeff)
                else:
                    color = pg.Color('gray85')

                self.drawCell(cell, color, True)
        
    def updateDataSpan(self):
        if self.updateDataSpan_ is None:
            self.minExposedDataValue = +np.inf
            self.maxExposedDataValue = -np.inf

            for j in range(self.grid.columnsNbr()):
                for i in range(self.grid.linesNbr()):
                    cell = self.grid.cellFromIdx([j,i])

                    if cell.elevation() < self.minExposedDataValue: self.minExposedDataValue = cell.elevation()
                    if cell.elevation() > self.maxExposedDataValue: self.maxExposedDataValue = cell.elevation()

            self.spanOfData = self.maxExposedDataValue - self.minExposedDataValue
        return self.updateDataSpan_

    def drawCells(self, cells, color = pg.Color('cadetblue1'), fillCell = True):
        for cell in cells:
            self.drawCell(cell, color, fillCell)

        pg.display.flip()
        self.processEvent()

    def drawCell(self, cell, color = pg.Color('black'), fillCell = False):
        if fillCell:
            pg.draw.rect(self.window, color, pg.Rect(cell.positionInScreenPixels(), self.grid.cellSizeInScreenPixels))
        else:
            pg.draw.rect(self.window, color, pg.Rect(cell.positionInScreenPixels(), self.grid.cellSizeInScreenPixels), 1)

    def drawLine(self, color, A, B):
        pg.draw.line(self.window, color, A.centerPositionInScreenPixels(), B.centerPositionInScreenPixels(), 2)

    def drawLines(self, color, cells):
        if len(cells) < 2: return

        for i in range(1, len(cells)):
            self.drawLine(color, cells[i - 1], cells[i])

    def printElevationField(self):
        for j in range(self.grid.columnsNbr()):
            for i in range(self.grid.linesNbr()):
                cell = self.grid.cellFromIdx([j,i])

                dataValueSTR = str(int(cell.elevation()))
                self.printCellData(cell, dataValueSTR)

    def printCellData(self, cell, dataSTR, board_thikness=1):
        font = pg.font.SysFont('timesnewroman', 12)
        text = font.render(dataSTR, True, pg.Color('black'), pg.Color('grey99'))
        textRect = text.get_rect()
        textRect.center = cell.centerPositionInScreenPixels()
        self.window.blit(text, textRect)

    def processEvent(self):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                self.display = False

            elif event.type == pg.KEYDOWN:
                if event.key == pg.K_SPACE:
                    self.updating = False

        while not self.updating: self.handSelection()
        if not self.display: pg.quit()

    def handSelection(self):
        for evnt in pg.event.get():
            if evnt.type == pg.QUIT:
                self.display = False
                self.updating = True

            if evnt.type == pg.MOUSEBUTTONDOWN:
                cell = self.grid.cellFromPositionInScreenPixels(pg.mouse.get_pos())

                if evnt.button == 1:
                    self.setAFromIdx(cell.Idx)

                if evnt.button == 3:
                    self.setBFromIdx(cell.Idx)

            elif evnt.type == pg.KEYDOWN:
                if evnt.key == pg.K_SPACE:
                    self.updating = True

    def windowCaption(self):
        self.iter = self.iter + 1
        return self.windowCaptionParte1 + ", iter: " + str(self.iter)

# display and mouse-select A and B points (A:captação:rightButton e B:entrega:leftButton)
displayManager = DisplayManager(grid)

#'A' cell selected,  [ 97 248]
#'B' cell selected,  [229  92]
displayManager.setAFromIdx([97, 248])
displayManager.setBFromIdx([229, 92])

#displayManager.setAFromIdx([0, -1])
#displayManager.setBFromIdx([-1, 0])

#displayManager.initiateDisplay()
#while displayManager.display:
#    displayManager.update()

In [ ]:
## Define Path ###

import math
def norm2Length(p1, p2):
    dislocation = np.array(p2) - np.array(p1)
    return math.sqrt(np.dot(dislocation.T, dislocation))
        
class Segment:
    def __init__(self, grid, cI, cF):
        self.grid = grid
        self.cI = cI
        self.cF = cF
        self.norm2Length = norm2Length(self.cI.centerPositionInMeters(), self.cF.centerPositionInMeters())
        self.profil = None

    def length(self):
        if self.profil is None: self.profil = SegmentProfil(self, self.grid)
        return self.profil.cumulativeLength()

    def maxRelativeElevation(self):
        if self.profil is None: self.profil = SegmentProfil(self, self.grid)
        return self.profil.maxRelativeElevation()

class SegmentProfil:
    def __init__(self, segment, grid):
        self.segment = segment
        self.grid = grid
        self.data = {'midPointsInMeters':[],
                     'relativeElevations':[],
                     'cumulativeLengths':[]}
        self.maxRelativeElevation_ = None
        self.process_()

    def midPointsInMeters(self):
        return self.data['midPointsInMeters']

    def relativeElevations(self):
        return self.data['relativeElevations']

    def maxRelativeElevation(self):
        if self.maxRelativeElevation_ is None:
            self.maxRelativeElevation_ = max(self.relativeElevations())
        return self.maxRelativeElevation_

    def cumulativeLengths(self):
        return self.data['cumulativeLengths']

    def cumulativeLength(self):
        return self.cumulativeLengths()[-1]

    def intersectedGridLinesIdxs_(self, A, B, axis):
        x0 = A[axis] #; print("x0 = ", x0)
        x1 = B[axis] #; print("x1 = ", x1)
        dx = x1-x0
        xMin = min(x0, x1)
        xMax = max(x0, x1)
        isign = int(np.sign(x1 - x0))
        v = [*range(int(xMin) + 1, int(xMax) + 1)] #; print("v = ", v)
        if isign == -1:
            v.reverse()
        return v

    def intersectionWithGridLines_(self, A, B, axis):
        v = []
        for i in self.intersectedGridLinesIdxs_(A, B, axis):
            ti = (i - A[axis]) / (B[axis] - A[axis]) #; print("ti = ", ti)
            xi = A[0] + ti * (B[0] - A[0]) #; print("xi = ", xi)
            yi = A[1] + ti * (B[1] - A[1]) #; print("yi = ", yi)
            v.append([ti, xi, yi])
        return v

    def process_(self):
        A = self.segment.cI.centerIdx #; print("A = ", A)
        B = self.segment.cF.centerIdx #; print("B = ", B)

        vi = self.intersectionWithGridLines_(A, B, 0) #; print("vi = ", vi)
        vj = self.intersectionWithGridLines_(A, B, 1) #; print("vj = ", vj)

        # # making the curve connecting intersection points [t, x, y]
        curve = [[0., A[0], A[1]]]
        curve.extend(vi)
        curve.extend(vj)
        curve.extend([[1., B[0], B[1]]])
        curve = np.array(curve)
        curve = curve[curve[:, 0].argsort()]
        curveInIdx = curve #; print("curveInIdx = ", curveInIdx)

        # deleting repeated points
        curve = [curveInIdx[0]]
        i = 1 
        while i < len(curveInIdx):
            if curveInIdx[i,0] - curveInIdx[i-1,0] < 10**-6:
                i = i+1
            curve.append(curveInIdx[i])
            i = i+1
        curveInIdx = curve #; print("curveInIdx = ", curveInIdx)

        # making the curve connecting intersection mid-points
        curve = [[0., A[0], A[1]]]
        for i in range(len(curveInIdx) - 1):
            curve.append((curveInIdx[i] + curveInIdx[i + 1]) / 2.)
        curve.append([1., B[0], B[1]])
        curveOfMidPointsInIdx = curve #; print("curveOfMidPointsInIdx = ", curveOfMidPointsInIdx)

        curve = []
        for point in curveOfMidPointsInIdx:
            curve.append(self.grid.positionInMetersFromIdx([point[1], point[2]]))
        self.data['midPointsInMeters'] = curve #; print("self.data['midPointsInMeters'] = ", self.data['midPointsInMeters'])

        lastPoint = self.data['midPointsInMeters'][0]
        cumulativeLength = 0.
        for point in self.data['midPointsInMeters']:
            self.data['relativeElevations'].append(point[2]-lastPoint[2])
            cumulativeLength = cumulativeLength + norm2Length(lastPoint, point)
            self.data['cumulativeLengths'].append(cumulativeLength)
            lastPoint = point

    def intersectedCells(self):
        cells = []
        for i in range(1, len(self.data['midPointsInMeters'])-1):
            cells.append(self.grid.cellFromPositionInMeters(self.data['midPointsInMeters'][i]))
        return cells

class Path:
    def __init__(self, grid, cellsIdx):
        self.grid = grid
        self.cellsIdx = cellsIdx
        
        self.cells = []
        for idx in self.cellsIdx:
            cell = self.grid.cellFromIdx(idx)
            self.cells.append(cell)

        self.segments = []
        for i in range(len(self.cells) - 1):
            segment = Segment(self.grid, self.cells[i], self.cells[i+1])
            self.segments.append(segment)

        # compute norm2Length
        self.norm2Length = 0.
        for segment in self.segments:
            self.norm2Length = self.norm2Length + segment.norm2Length

        self.length_ = None
        self.maxRelativeElevation_ = None

    def length(self):
        if self.length_ is None:
            # compute length_
            self.length_ = 0.
            for segment in self.segments:
                self.length_ = self.length_ + segment.length()
        return self.length_

    def maxRelativeElevation(self):
        if self.maxRelativeElevation_ is None:
            # compute maxRelativeElevation_
            self.maxRelativeElevation_ = 0.
            for segment in self.segments:
                self.maxRelativeElevation_ = max(self.maxRelativeElevation_, segment.maxRelativeElevation())
        return self.maxRelativeElevation_

# print the length of the straight path, from A to B
ABPath = Path(grid, grid.cellsIdx([grid.A, grid.B]))
print("ABPath.norm2Length = ", ABPath.norm2Length)
print("ABPath.length() = ", ABPath.length())

In [ ]:
### Define the Cost Manager ###

# Despesa anual com tubos = Kt * (A0 + A1 * D + A2 * (D ^ 2)) * L
# E, para esse material, A0 = 5.8502; A1 = 87.2802 e A2 = 223.9541
# Para se calcular a despesa annual com tubos, temos que annualizar o custo
# do investimento para poder somar com os custos anuais da energia.
# Para isso calculamos o parâmetro de annualização kt (fórmula em linguagem BASIC):
# Kt = C1 * JUROS * (((1 + JUROS) ^ T) / (((1 + JUROS) ^ T) - 1))
# T = vida útil do empreendimento ou tempo de retorno do capital investido na aquisição dos tubos dado em anos
tubParameters = {
    'diameter':0.6, #[m]
    'A0':5.8502,
    'A1':87.2802,
    'A2':223.9541,

    'c1':5.9333,
    'interests': 0.06, # [-]
    'T':50., # [anos]
}

# perdas de carga ao longo do comprimento da adutora.
# Foi utilizada para a aplicação a fórmula de Hazen-Williams com coeficiente de atrito C = 140.
# A Fórmula da perda de carga unitária (PU) é (em linguagem Basic)
# PU = 10.643 * (Q ^ 2.85) / (C ^ 1.85) / (D ^ 4.87)
# a altura manométrica Hm é a soma do desnível geométrico
# com as perdas de carga ao longo do comprimento da adutora.

# P = 9.8 * Q * Hm / Neta
# P: Potência kW; 
# Q = Vazão em m3/s;
# Hm = altura manométrica (mca);
# Neta = rendimento do conjunto motor-bomba (%, entra na fórmula dividido por 100);

# Despesa annual com energia = P * NHBA * Tarifa
# NHBA = número de horas de funcionamento annual do sistema;
# Tarifa = custo da energia elétrica (R$/kWh);
energyParameters = {
    'flowRate':0.351, # [m^3/s]
    'frictionCoeff':140.,
    'diameter':tubParameters['diameter'],
    'pumpPerformance':0.85, # [-]

    'annualWorkload':7300, # [horas/ano]
    'tariff':0.31, # [real/KWh]
}

projectParameters = {'tubParameters':tubParameters, 'energyParameters':energyParameters}

# print project parameters
import pprint
pprint.pprint(projectParameters)

class CostManager:
    def __init__(self, projectParameters):
        self.projectParameters = projectParameters
        self.tubParameters = self.projectParameters['tubParameters']
        self.energyParameters = self.projectParameters['energyParameters']

        # Despesa annual com tubos = Kt * (A0 + A1 * D + A2 * (D ^ 2))
        # Kt = C1 * JUROS * (((1 + JUROS) ^ T) / (((1 + JUROS) ^ T) - 1))
        C1 = self.tubParameters['c1']
        JUROS = self.tubParameters['interests']
        T = self.tubParameters['T']
        Kt = C1 * JUROS * (((1 + JUROS)**T) / (((1 + JUROS)**T) - 1))
        A0 = self.tubParameters['A0']
        A1 = self.tubParameters['A1']
        A2 = self.tubParameters['A2']
        D = self.tubParameters['diameter']
        self.tubAnnualUnitCost = Kt * (A0 + A1 * D + A2 * (D**2)) # [real/m]

    # gageHeight: desnível geométrico, maior diferença de cota
    # a ser vencida entre a captação e a entrega
    def gageHeight(self, path): # [m]
        return path.maxRelativeElevation()

    # perdas de carga ao longo do comprimento da adutora.
    # Foi utilizada para a aplicação a fórmula de Hazen-Williams com coeficiente de atrito C = 140.
    # A Fórmula da perda de carga unitária (PU) é (em linguagem Basic)
    # PU = 10.643 * (Q ^ 2.85) / (C ^ 1.85) / (D ^ 4.87)
    def heightLosses(self, path): # [m]
        Q = self.energyParameters['flowRate']
        C = self.energyParameters['frictionCoeff']
        D = self.energyParameters['diameter']
        PU = 10.643 * (Q**2.85) / (C**1.85) / (D**4.87)
        return PU * path.length()

    # a altura manométrica Hm é a soma do desnível geométrico
    # com as perdas de carga ao longo do comprimento da adutora.
    def manometricHead(self, path): # [mca]
        return self.gageHeight(path) + self.heightLosses(path)

    # P = 9.8 * Q * Hm / Neta
    # P: Potência kW; 
    # Q = Vazão em m3/s;
    # Hm = altura manométrica (mca);
    # Neta = rendimento do conjunto motor-bomba (%, entra na fórmula dividido por 100);
    def potency(self, path): # [kW]
        Q = self.energyParameters['flowRate']
        Hm = self.manometricHead(path)
        Neta = self.energyParameters['pumpPerformance']
        return 9.8 * Q * Hm / Neta

    # Despesa annual com energia = P * NHBA * Tarifa
    # NHBA = número de horas de funcionamento annual do sistema;
    # Tarifa = custo da energia elétrica (R$/kWh).
    def energyAnnualCost(self, path):
        P = self.potency(path)
        NHBA = self.energyParameters['annualWorkload']
        Tarifa = self.energyParameters['tariff']
        return P * NHBA * Tarifa

    def tubAnnualCost(self, path):
        return self.tubAnnualUnitCost * path.length()

    def totalAnnualCost(self, path):
        return self.tubAnnualCost(path) + self.energyAnnualCost(path)
    
    def roughAnnualCost(self, path):
        return self.tubAnnualUnitCost * path.norm2Length

costManager = CostManager(projectParameters)
print("roughAnnualCost = ", costManager.roughAnnualCost(ABPath))
print("totalAnnualCost = ", costManager.totalAnnualCost(ABPath))

In [ ]:
### Define the Solver ###

import timeit, time, datetime
class Solver:
    def __init__(self, displayManager, costManager):
        self.displayManager = displayManager
        self.displayManager.setSolver(self)
        self.grid = self.displayManager.grid
        self.A = self.displayManager.A
        self.B = self.displayManager.B
        self.costManager = costManager

    def solve(self):
        print("Simulation start: Searching the best path from A to B...\n")
        simStartInstant = timeit.default_timer()

        mintotalAnnualCost = self.costManager.totalAnnualCost(Path(self.grid,[self.A.Idx,self.B.Idx]))
        self.selectedMinCosts = [mintotalAnnualCost]
        self.bestPathCells = [self.A]
        self.iterIdx = 0
        print("iter. ", self.iterIdx,": ", self.bestPathCells[-1].Idx, " ", "{:.6f}".format(self.selectedMinCosts[-1]))

        while True:
            self.updateCMs()
            mintotalAnnualCost = +np.inf

            for CM in self.CMs:
                if CM == self.currentCell:
                    continue
    
                twoSegmentsPath = Path(self.grid, [self.currentCell.Idx, CM.Idx, self.B.Idx])
                roughAnnualCost = self.costManager.roughAnnualCost(twoSegmentsPath)
                if self.costManager.roughAnnualCost(twoSegmentsPath) > mintotalAnnualCost:
                    continue

                totalAnnualCost = self.costManager.totalAnnualCost(twoSegmentsPath)
                if totalAnnualCost < mintotalAnnualCost:
                    mintotalAnnualCost = totalAnnualCost
                    selectedCM = CM

            self.bestPathCells.append(selectedCM)
            self.selectedMinCosts.append(mintotalAnnualCost)
            self.iterIdx = self.iterIdx + 1
            print("iter. ", self.iterIdx,": ", self.bestPathCells[-1].Idx, " ", "{:.6f}".format(self.selectedMinCosts[-1]))

            if self.bestPathCells[-1] == self.B:
                deltaT = timeit.default_timer() - simStartInstant
                print("\nbestPathtotalAnnualCost = ", "{:.0f}".format(solver.bestPathtotalAnnualCost()))
                print("The simulation took: ", datetime.timedelta(seconds = deltaT)," (hh:mm:ss)")
                return
    
    def updateCMs(self):
        self.updateCMOfReference()
        
        A = self.CMOfReference
        column = grid.columnFromCell(A); line = grid.lineFromCell(A)
        columnIdx = A.columnIdx(); lineIdx = A.lineIdx()

        self.CMs = [A]
        leftSemiColumn = column[:lineIdx]; self.CMs.extend(leftSemiColumn)
        leftSemiLine = line[:columnIdx]; self.CMs.extend(leftSemiLine)
        rightSemiColumn = column[lineIdx+1:]; self.CMs.extend(rightSemiColumn)
        rightSemiLine = line[columnIdx+1:]; self.CMs.extend(rightSemiLine)
        
    def updateCMOfReference(self):
        self.currentCell = self.bestPathCells[-1]
        
        deltaJ = np.sign(self.B.columnIdx() - self.currentCell.columnIdx())
        nextColumnIdx = self.currentCell.columnIdx() + deltaJ
        
        deltaI = np.sign(self.B.lineIdx() - self.currentCell.lineIdx())
        nextLineIdx = self.currentCell.lineIdx() + deltaI
        
        self.CMOfReference = self.grid.cellFromIdx([nextColumnIdx, nextLineIdx])
        #print('self.CMOfReference.Idx = ' , self.CMOfReference.Idx)

    def totalAnnualCost(self, path):
        return self.costManager.totalAnnualCost(path)

    def bestPathtotalAnnualCost(self):
        return self.totalAnnualCost(self.bestPath())

    def bestPath(self):
        return Path(self.grid, self.grid.cellsIdx(self.bestPathCells))

solver = Solver(displayManager, costManager)
solver.solve()

In [ ]:
### Display Solution ###
displayManager.initiateDisplay()
while displayManager.display:
    displayManager.update()